In [4]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
# sys.path.append("..")
sys.path.append("/home/yifan/projects/cophi/ContraVis")
import numpy as np

sys.path.append("/home/yifan/projects/deepdebugertool/DLVisDebugger")
# REF_PATH = "/home/yifan/Exp/Noise/0.2/experiment_vis"
# CONFIG_PATH = "/home/yifan/experiments/noise/20"
# TAR_PATH = "/home/yifan/dataset/cleanfornoise10"

# # CLEAN_PATH = "/home/yifan/Exp/Noise/0.2/experiment1"
TAR_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"



ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis2'

########## initulize reference data and target data
from singleVis.DataInit import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
DEVICE = "cuda:1"
tar_datainit = DataInit(TAR_PATH,TAR_PATH,TAR_EPOCH,DEVICE)
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH,DEVICE)

ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()




NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 7286.23it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 6697.21it/s]


In [5]:
from scipy.linalg import svd  
def orthogonal_mapping(data1, data2):
        """
        use Orthogonal Mapping, map data2 to data1's space。
    
        :param data1: numpy array, shape (n_samples, n_features)
        :param data2: numpy array, shape (n_samples, n_features)
        :return: data2_mapped: numpy array, mapped data2
        """
        # step1: Centralized data
        data1_centered = data1 - np.mean(data1, axis=0)
        data2_centered = data2 - np.mean(data2, axis=0)
        # step2: Calculate the cross-covariance matrix
        C = data2_centered.T @ data1_centered
        # step3: singular value decomposition
        U, _, Vt = svd(C)

        # step4: Compute orthogonal mapping matrix
        W = U @ Vt

        # step5: Apply mapping matrix
        data2_mapped = data2_centered @ W
    
        return data2_mapped
X_train1 = ref_provider.train_representation(REF_EPOCH)
X_train2 = tar_provider.train_representation(TAR_EPOCH)
X_train2 = X_train2.reshape(X_train2.shape[0],X_train2.shape[1])
print(X_train1.shape)
print(X_train2.shape)
data2_mapped = orthogonal_mapping(X_train1,X_train2)

(50000, 512)
(50000, 512)


In [7]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(data2_mapped, X_train2)
print(rate)

0.9999986666666667


In [4]:
torch.cuda.empty_cache()

In [6]:
from contrast.transfomration import TransformationTrainer

trainer = TransformationTrainer(X_train1,X_train2, DEVICE)
tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train_advanced()
# tarns_model,tar_mapped,ref_reconstructed  = trainer.transformation_train()

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [5]:
ref_pred = ref_provider.get_pred(REF_EPOCH, X_train1).argmax(axis=1)
tar_pred = tar_provider.get_pred(TAR_EPOCH, X_train2).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 7079.62it/s]


In [5]:
from eval.evaluate import *
rate = evaluate_high_dimesion_trans_knn_preserving(tar_mapped, X_train2)
print(rate)

0.49024933333333337


In [12]:

reconstructed_pred = tar_provider.get_pred(TAR_EPOCH, ref_reconstructed).argmax(axis=1)

100%|██████████| 250/250 [00:00<00:00, 6842.17it/s]


In [13]:
diff = 0
for i in range(len(reconstructed_pred)):
    if reconstructed_pred[i] != tar_pred[i]:
        diff = diff+1
print(diff)

3


In [14]:
from sklearn.neighbors import NearestNeighbors

sub_len = 50000

# 1. use cos sim build 
model = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='auto').fit(tar_mapped[:sub_len])

# 2. find X_train1 most sim point data2_mapped中的最相似点
distances, most_similar_points = model.kneighbors(X_train1[:sub_len])

most_similar_points = most_similar_points.flatten()  # 将索引数组转为1D

# 3. 比较预测值
m = np.sum(ref_pred[:sub_len] != tar_pred[most_similar_points])
k = np.sum(ref_pred[:sub_len] != tar_pred[:sub_len])

print("diff pred number is {}".format(m))
print("index diff pred number is {}".format(k))


diff pred number is 41
index diff pred number is 12
